# C. Perform MintPy Time Series Analysis

*Author: Alex Lewandowski; Alaska Satellite Facility*

This notebook assumes that you have loaded a time series into MintPy and have prepared a custom configuration file.


**MintPy Workflow**
<p align="left">
  <img width="600" src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/notebook_images/MintPyWorkflow.jpg">
</p>     
<p style="text-align: center;">
    (Figure from Yunjun et al., 2019)
</p>

- Blue Ovals: correcting unwrapping errors and inverting for the raw phase time-series
- Green Ovals: correcting for noise from different sources to obtain the displacement time-series
- Dashed Ovals: Optional Steps

---

<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">
      <img src="github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">opensarlab_MintPy_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, ASF data access, or performing SBAS time series analyses with MintPy?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---

## 0. Import Required Software

In [ ]:
from pathlib import Path
import shutil
import sys

import h5py
from ipyfilechooser import FileChooser
from mintpy.cli import view, tsview, plot_network, plot_transection
import mintpy.utils
from mintpy.utils import plot, readfile
import numpy as np
import opensarlab_lib as osl

current = Path("..").resolve()
sys.path.append(str(current))
import util.util as util

---
## 1. Select your project's custom config file

- This is located in your project's `MintPy` directory
- It is a text file named after your project
  - `path/to/MinPy/my_project.txt`

In [ ]:
file_chooser_path = util.get_recent_mintpy_config_path()
if file_chooser_path:
    fc = FileChooser(path=file_chooser_path.parent, filename=file_chooser_path.name, select_default=True)
else:
    file_chooser_path = Path.home()
    fc = FileChooser(file_chooser_path, select_default=False)
    
print("Select your custom MintPy config file (MintPy/my_project_name.txt):")
display(fc)

In [ ]:
if Path(fc.selected) != file_chooser_path:
    util.write_recent_mintpy_config_path(Path(fc.selected))

config_path = Path(fc.selected)
mint_path = config_path.parent
inputs_path = mint_path / 'inputs'
ifgram_path = inputs_path / 'ifgramStack.h5'
plot_path = mint_path / 'plots'
velocity_path = mint_path/'velocity.h5'

---
## 2. Modify the Network

**Run the `modify_network` step**

- Identifies and excludes interferograms (i.e. affected by remaining coherence phase-unwrapping errors) before the network inversion
- Updates metadata with changes to your custom config

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep modify_network

---
## 3. Plot the interferogram network

Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- `ifgramStack_coherence_spatialAvg.txt`: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- `Network.pdf`: Displays the network of interferograms on time-baseline coordinates, colorcoded by average coherence of the interferograms. 
- `CoherenceMatrix.pdf` shows the average coherence pairs between all available pairs in the stack.

In [ ]:
%matplotlib inline
with osl.work_dir(mint_path):
    scp_args = f'{inputs_path}/ifgramStack.h5 --cmap-vlist 0.2 0.7 1.0'
    plot_network.main(scp_args.split())
    plots = ['pbaseHistory.pdf', 'coherenceHistory.pdf', 'coherenceMatrix.pdf', 'network.pdf']
    for p in plots:
        if (mint_path/p).exists():
            (mint_path/p).rename(f'{plot_path}/{p}')

---
## 4. Set the Reference Point

**Run the `reference_point` step**

The interferometric phase is a relative observation by nature. The phases of each unwrapped interferogram are relative with respect to an arbitrary pixel. Therfore, we need to reference all interferograms to a common reference pixel.

The `reference_point` step selects a common reference pixel for the stack of interferograms. The default approach of MintPy is to choose a pixel with the highest spatial coherence in the stack. Other options include specifying the longitude and latitude of the desired reference pixel or the line and column number of the refence pixel. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_point

**Confirm presence of expected output files from `reference_point` step**

In [ ]:
spatial_coh_pth = mint_path / 'avgSpatialCoh.h5'
conn_comp_mask_path = mint_path / 'maskConnComp.h5'

if not spatial_coh_pth.exists():
    raise Exception(f'Possible error running smallbaselineApp reference_point step. Expected output not found: {spatial_coh_pth}')
if not conn_comp_mask_path.exists():
    raise Warning(f'Possible error running smallbaselineApp reference_point step. Ignore if you are using HyP3 INSAR_GAMMA data, which does not include connected components. Expected output not found: {conn_comp_mask_path}')

**Running the "reference_step" adds additional attributes "REF_X, REF_Y" and "REF_LON, REF_LAT" to the `ifgramStack.h5` file. To see the attributes of the file run `info.py`**

In [ ]:
!info.py $inputs_path/ifgramStack.h5 | egrep 'REF_'

---
## 5. Run a Quick Overview

**Run the `quick_overview` step**

- Assess possible groud deformation using the velocity from traditional interferogram stacking 
    - *reference: Zebker et al. (1997, JGR)*
- Assess distribution of phase unwrapping error from the number of interferogram triplets with non-zero integer ambiguity of closure phase 
    - *reference: T_int in Yunjun et al. (2019, CAGEO). Related to section 3.2, equation (8-9) and Fig. 3d-e.*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep quick_overview

**Confirm presence of expected output files from `quick_overview` step**

In [ ]:
avg_phase_vel_path = mint_path / 'avgPhaseVelocity.h5'

if not avg_phase_vel_path.exists():
    raise Exception(f'Possible error running smallbaselineApp quick_overview step. Expected output not found: {avg_phase_vel_path}')

---
## 6. Perform Phase Unwrapping Error Correction (Optional)

### Unwrapping Error Correction can be performed on `HYP3_ISCE_BURST` data but not on `HYP3_GAMMA` data
- Gamma processing software does not produce the connected components data required for this step

### To perform unwrapping error correction, you must use one of the following `mintpy.unwrapError.method` configuration options:
-  `bridging`
    - suitable for regions separated by narrow decorrelated features, e.g. rivers, narrow water bodies 
-  `phase-closure`
    - suitable for highly redundant network 
-  `bridging+phase_closure`
    -  recommended when there is a small percentage of errors left after bridging

### It will likely be necessary to set a `mintpy.unwrapError.connCompMinArea` value lower than the default of $2.5e3$ pixels
- Connected components containing fewer pixels than the minimum will be removed
- To determine a suitable `mintpy.unwrapError.connCompMinArea`:
  1. Select a minimum connected components size for your dataset in m<sup>2</sup>
      - This area represents the minimum size area in which you expect to encounter actual surface deformation.
      - Any connected components smaller than this area will be removed as noise.  
  2. Determine the area covered by a pixel (Resolution<sup>2</sup>)
  3. Calculate `connCompMinArea`:
      - $\texttt{connCompMinArea} = \frac{\texttt{min\_area} \text{ m}^2}{\texttt{resolution}^2}$

### INSAR_ISCE_BURST: Resolution by Number of Looks
<table style="margin-left: 0;">
<tr>
    <th>Looks</th>
    <th>Resolution</th>
</tr>
<tr>
    <td>20x4</td>
    <td>160 m</td>
</tr>
<tr>
    <td>10x2</td>
    <td>80 m</td>
</tr>
<tr>
    <td>1x5</td>
    <td>40 m</td>
</tr>
</table>

https://hyp3-docs.asf.alaska.edu/guides/burst_insar_product_guide/

**Visually examine the unwrapped interferograms for potential phase unwrapping errors**

In [ ]:
%matplotlib inline

ifg_vmin, ifg_vmax = util.get_mintpy_vmin_vmax(ifgram_path, bottom_percentile=0.05)
ifg_vmin /= 100
ifg_vmax /= 100
print(f"ifg_vmin: {ifg_vmin}, ifg_vmax: {ifg_vmax}")

args = f'{ifgram_path} unwrapPhase-* -v {ifg_vmin} {ifg_vmax} --zero-mask --noaxis --noverbose --figsize 12 50 --no-tight-layout'
view.main(args.split())

**Examine the number of interferogram triples with non-zero phase closure**

For an interferogram triplet ($\Delta\phi^{ij}$, $\Delta\phi^{jk}$ and $\Delta\phi^{ik}$), unwrapping errors will introduce a non-zero integer component $C_{int}^{ijk}$ in the closure phase $C^{ijk}$. Therefore, the number of interferogram triplets with non-zero integer ambiguity $T_{int}$ can be used to detect unwrapping errors (Yunjun et al., 2019):

$$ \large C^{ijk}=\Delta\phi^{ij}+\Delta\phi^{jk}-\Delta\phi^{ik}$$

$$ \large C_{int}^{ijk}=\frac{C^{ijk}-wrap(C^{ijk})}{2\pi}$$

$$ \large T_{int}=\sum_{i=1}^T(C_{int}^{ijk}!=0)$$

where $warp$ is an operator to wrap the input number into $[-\pi, \pi)$; $T$ is the number of interferogram triplets.

**Plot the histogram for the number of triplets with non-zero integer ambiguity.**

Take home messages from $T_{int}$ map and histogram:

1. Areas with $T_{int}$ > 0 have unwrapping errors.
2. Areas sharing the common positive $T_{int}$ value could be corrected.
3. Areas with wide-distributed $T_{int}$ value indicates random unwrapping errors, which are difficult to correct.

(https://github.com/parosen/Geo-SInC/blob/main/EarthScope2023/5.2_Intro_to_MintPy/smallbaselineApp_aria.ipynb)

In [ ]:
%matplotlib inline

with osl.work_dir(mint_path):
    !smallbaselineApp.py {config_path} --dostep quick_overview
    plot.plot_num_triplet_with_nonzero_integer_ambiguity(mint_path/'numTriNonzeroIntAmbiguity.h5', disp_fig=True, fig_size=[14, 4])

**Confirm that an unwrapping error method was not defined in the config if working with `HYP3_GAMMA` data**

- `HYP3_GAMMA` products do not include required connected components data
- Setting a `mintpy.unwrapError.method` when working with `HYP3_GAMMA` data will cause the inversion step to fail, as expected `unwrapPhase_bridging` and/or `unwrapPhase_phaseClosure` datasets will be missing
- Check and (if needed) correct this situation in the cell below

In [ ]:
config_dict = readfile.read_template(config_path)
hyp3_isce_burst = 'mintpy.load.connCompFile' in config_dict.keys()
unwrap_error_correct = 'mintpy.unwrapError.method' in config_dict.keys() and config_dict['mintpy.unwrapError.method'] not in ['no', 'auto']

if unwrap_error_correct and not hyp3_isce_burst:
    config_dict['mintpy.unwrapError.method'] = 'no'
    config_str = ""
    for k, v in config_dict.items():
        config_str += f'{k} = {v}\n'
    config_path.write_text(config_str)  
    unwrap_error_correct = False

**Perform the `correct_unwrap_error` step**

In [ ]:
# Delete previous results if attempting to rerun correct_unwrap_error step
# Step is skipped if output data are present
try:
    with h5py.File(ifgram_path, 'a') as f:
        del f['unwrapPhase_phaseClosure']
except:
    pass
try:
    with h5py.File(ifgram_path, 'a') as f:
        del f['unwrapPhase_bridging']
except:
    pass

if unwrap_error_correct:
    with osl.work_dir(mint_path):
        !smallbaselineApp.py {config_path} --dostep correct_unwrap_error

**Confirm presence of expected output dataset/s from `correct_unwrap_error` step**

In [ ]:
if unwrap_error_correct:
    unw_err_ds = []
    if 'bridging' in config_dict['mintpy.unwrapError.method']:
        unw_err_ds.append('unwrapPhase_bridging')
    if 'phase_closure' in config_dict['mintpy.unwrapError.method']:
        unw_err_ds.append('unwrapPhase_phaseClosure')
    with h5py.File(ifgram_path, 'r') as f:
        for ds in unw_err_ds:
            if ds not in f.keys():
                raise Exception(f'Possible error running smallbaselineApp correct_unwrap_error step. Expected output dataset ({ds}) not found in  {ifgram_path}')

**Plot the interferograms after unwrapping error correction**

In [ ]:
%matplotlib inline
if unwrap_error_correct:
    print("This may take some time and appear to show a blank plot while all the subplots load.\n")
    unwrapp_args = f'{ifgram_path} unwrap* --zero-mask --figsize 12 50 --noverbose --no-tight-layout'
    view.main(unwrapp_args.split())

**Plot the connected components**

In [ ]:
%matplotlib inline
if unwrap_error_correct:
    print("This may take some time and appear to show a blank plot while all the subplots load.\n")
    unwrapp_args = f'{ifgram_path} conn* --zero-mask --figsize 12 50 --noverbose --no-tight-layout'
    view.main(unwrapp_args.split())

---
## 7. Inverting the Small Baseline network

**Run the `invert_network` step**

- Invert the network of differential unwrapped interferograms to estimate the time-series of unwrapped phase with respect to a reference acquisition date
- By default mintpy selects the first acquisition
- The estimated time-series is converted to distance change from radar to target and is provided in meters.

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep invert_network

**Confirm presence of expected output files from `invert_network` step**

In [ ]:
ts_path = mint_path / 'timeseries.h5'
temp_coh_path = mint_path / 'temporalCoherence.h5'
num_inv_ifgram_path = mint_path / 'numInvIfgram.h5'

for pth in [ts_path, temp_coh_path, num_inv_ifgram_path]:
    if not pth.exists():
        raise Exception(f'Possible error running smallbaselineApp invert_network step. Expected output not found: {pth}')

**Plot the inverted time series**

In [ ]:
%matplotlib inline
inv_vmin, inv_vmax = util.get_mintpy_vmin_vmax(ts_path, mask_path=mint_path/'maskTempCoh.h5', bottom_percentile=0.0)

# uncomment the following 2 lines to manually set the vmin and vmax values
# inv_vmin = -2.0
# inv_vmax = 2.0

scp_args = f'{ts_path} --notick --noaxis -v {inv_vmin} {inv_vmax} --dpi 600 --figsize 15 15'
view.main(scp_args.split())

---
## 8. Correct for Tropospheric Propagation Delays (Optional)

**Run the `correct_troposphere` step**

***This will be skipped if `mintpy.troposphericDelay.method` is set to "no" in your custom config***

- Requires configuring a `.cdsapirc` file for CDS access
  - Run the `CDS_Access.ipynb` (Set Up Climate Data Store Access) Notebook
- Uses ECMWF [ERA5 climate reanalysis pressure data](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&keywords=((%20%22Product%20type:%20Reanalysis%22%20)%20AND%20(%20%22Provider:%20Copernicus%20C3S%22%20))&text=pressure)

In [ ]:
template = mintpy.utils.readfile.read_template(config_path)

tropo = ('mintpy.troposphericDelay.method' in template.keys() and template['mintpy.troposphericDelay.method'] != 'no') or \
'mintpy.troposphericDelay.method' not in template.keys()

era5_path = mint_path / 'ERA5'
timeseries_era5_path = mint_path / 'timeseries_ERA5.h5'
inputs_era5_path = mint_path / 'inputs/ERA5.h5'

# Uncomment code below to delete old tropospheric data if it exists,
# allowing MintPy to rerun tropospheric_correction
# for f in [timeseries_era5_path, inputs_era5_path]:
#     try:
#         f.unlink()
#     except FileNotFoundError:
#         pass
# try:
#     shutil.rmtree(era5_path)
# except FileNotFoundError:
#     pass
    
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_troposphere

In [ ]:
if tropo:
    for pth in [era5_path, timeseries_era5_path, inputs_era5_path]:
        if not pth.exists():
            raise Exception(f'Possible error running smallbaselineApp correct_troposphere step. Expected output not found: {pth}')

---
## 9. Phase Deramping (Optional)

**Run the `deramp` step**

***This will be skipped if `mintpy.deramp` is set to "auto" or "no" in your custom config***

>"Estimate and remove a phase ramp for each acquisition based on the reliable pixels.
>
>Recommended for localized deformation signals, i.e. volcanic deformation, landslide and land subsidence, etc.
>
>NOT recommended for long spatial wavelength deformation signals, i.e. co-, post- and inter-seimic deformation."
>
> ***https://github.com/insarlab/MintPy/blob/4bbca8c531ee021e64112da0e7885959b222a652/src/mintpy/defaults/smallbaselineApp.cfg#L253-L258***

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep deramp

In [ ]:
deramp = ('mintpy.deramp' in template.keys() and template['mintpy.deramp'] not in ['no', 'auto']) or \
'mintpy.deramp' not in template.keys()

if deramp and len(list(mint_path.glob('timeseries_*ramp.h5'))) < 1:
    raise Exception(f"Possible error running smallbaselineApp deramp step. Expected output of either {mint_path/'timeseries_ramp.h5'} or {mint_path/'timeseries_ERA5_ramp.h5'}")

---
## 10. Correct for DEM Errors

**Run the `correct_topography` step**

- MintPy will use the input file `timeseries.h5` if tropospheric correction is turned off in the config
- MintPy will use the input file `timeseries_ERA5.h5` if tropospheric correction is turned on in the config

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep correct_topography

In [ ]:
if len(list(mint_path.glob('timeseries_*demErr.h5'))) < 1:
    raise Exception(("Possible error running smallbaselineApp correct_topography step.\n"
                     "Expected one of the following outputs:\n"
                     f"- {mint_path/'timeseries_demErr.h5'}\n"
                     f"- {mint_path/'timeseries_ERA5_demErr.h5'}\n"
                     f"- {mint_path/'timeseries_ramp_demErr.h5'}\n"
                     f"- {mint_path/'timeseries_ERA5_ramp_demErr.h5'}"))

## 11. Calculate the Root Mean Square (RMS) of Residual Phase Time-Series for Each Acquisition

**Run the `residual_RMS` step**

- *reference: Yunjun et al. (2019, section 4.9 and 5.4)*
- To remove the long wavelength component in space, a phase ramp is removed for each acquisition
- Sets optimal reference date to date with min RMS
- Sets exclude dates (outliers) to dates with RMS > cutoff * median RMS (Median Absolute Deviation)

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep residual_RMS

## 12. Reference the Entire Time-Series to One Date in Time

**Run the `reference_date` step**

- *reference: Yunjun et al. (2019, section 4.9)*

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep reference_date

## 13. Estimate The Long-Term Velocity Rate

**Run the `velocity` step**

The timeseries file contains three datasets:
- the `time-series` dataset, which is the interferometric range change for each acquisition relative to the reference acquisition
- the `date` dataset, which contains the acquisition date for each acquisition
- the `bperp` dataset, which contains the timeseries of the perpendicular baseline 

The ground deformation caused by many geophysical or anthropogenic processes are linear at first order approximation. Therefore it is common to estimate the rate of the ground deformation which is the slope of linear fit to the time-series. 

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep velocity

In [ ]:
vel_path = mint_path / 'velocity.h5'

if not vel_path.exists():
    raise Exception(f'Possible error running smallbaselineApp velocity step. Expected output not found: {vel_path}')

**Plot the velocity map**

Note: The min/max values in this plot should be adjusted to your data set. 

The output of the following call will show a data range. You can run the cell once to collect this information and re-run it after adjustments are made.

- You should update `vmin` and `vmax` to match your data range
- The `vmin` and `vmax` values that you assign will be used for additional plots in this notebook
- If you wish to set 0 deformation to the center of the color scale (green), adjust your min/max values so they are symmetrical
    - For example, given a data range of `[-10.691118, 13.904866]`, appropriate symmetrical min/max values would be `[-14.0, 14.0]` 

In [ ]:
colormap_select = osl.select_parameter(
    [
        "Display red for uplift, and blue for subsidence",
        "Display blue for uplift, and red for subsidence"
    ]
)
display(colormap_select)

In [ ]:
%matplotlib inline
cmap = 'jet' if 'red for uplift' in colormap_select.value else 'jet_r'

vel_vmin, vel_vmax = util.get_mintpy_vmin_vmax(mint_path/'velocity.h5', mask_path=mint_path/'maskTempCoh.h5', bottom_percentile=0.05)

# uncomment the following 2 lines to manually set the vmin and vmax values
# vel_vmin = -2.0
# vel_vmax = 2.0

scp_args = f'{mint_path}/velocity.h5 velocity -v {vel_vmin} {vel_vmax} --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity.png --update --colormap {cmap}'
view.main(scp_args.split())

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

---
## 14. Geocode velocity.h5 in Preparation for Creating a velocity.kmz

**Run the `geocode` step**

- This is unnecessary for geocoded HyP3 data but would be needed for non-geocoded data

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep geocode

---
## 15. Create a kmz File

**Run the `google_earth` step**

In [ ]:
!smallbaselineApp.py $config_path --work-dir {mint_path} --dostep google_earth

In [ ]:
vel_kmz_path = mint_path / 'velocity.kmz'

if not vel_path.exists():
    raise Exception(f'Possible error running smallbaselineApp google_earth step. Expected output not found: {vel_kmz_path}')

---
## 16. Plot the unwrapped inverted timeseries again after tropospheric (optional) and topospheric correction

In [ ]:
geotiff_path = mint_path / 'GeoTiffs'
disp_path = geotiff_path / 'displacement_maps'
unwrapped_path = disp_path / 'unwrapped'
unwrapped_path.mkdir(parents=True, exist_ok=True)
ts_demErr_path = list(mint_path.glob('timeseries*_demErr.h5'))

**If you have both tropospheric delay corrected and uncorrected time series available, select which to plot**

- If you have run the time series more than once, with tropospheric correction both on and off, you will have time series saved for both configurations
- `timeseries_demErr.h5` is the time series not corrected for tropospheric delay
- `timeseries_ERA5_demErr.h5` is the time series corrected for tropospheric delay

In [ ]:
if len(ts_demErr_path) > 1:
    ts_select_option = osl.select_parameter(
        [
            ts_demErr_path[0],
            ts_demErr_path[1]
        ],
        description='Select the time series file you wish to plot:'
    )
    display(ts_select_option)
else:
    ts_demErr_sel_path = ts_demErr_path[0]

In [ ]:
if len(ts_demErr_path) > 1:
    ts_demErr_sel_path = ts_select_option.value

inv_vmin, inv_vmax = util.get_mintpy_vmin_vmax(ts_demErr_sel_path, mask_path=mint_path/'maskTempCoh.h5',  bottom_percentile=0.0)

# uncomment the following 2 lines to manually set the vmin and vmax values
# inv_vmin = -2.0
# inv_vmax = 2.0

scp_args = f'{ts_demErr_sel_path} --notick --noaxis -v {inv_vmin} {inv_vmax} --dpi 600 --figsize 15 15 --outfile {unwrapped_path}/unwrapped_inverted_ts.png'
view.main(scp_args.split())

---
## 17. Plot a Motion Transect

**Select two points to define a transect on an interactive plot**

In [ ]:
%matplotlib widget

vel_vmin, vel_vmax = util.get_mintpy_vmin_vmax(velocity_path, mask_path=mint_path/'maskTempCoh.h5',  bottom_percentile=0.05)

# uncomment the following 2 lines to manually set the vmin and vmax values
# vel_vmin = -2.0
# vel_vmax = 2.0

data, vel_info = mintpy.utils.readfile.read(velocity_path)
mask = np.ma.masked_where(data==0, data)
data = mask.filled(fill_value=np.nan)
line = osl.LineSelector(data, figsize=(9, 9), cmap='jet', vmin=vel_vmin/100, vmax=vel_vmax/100)

**Plot the selected motion transect**

In [ ]:
from IPython.display import Markdown

geotrans = (float(vel_info['X_FIRST'])), float(vel_info['X_STEP']) , 0.0, float(vel_info['Y_FIRST']), 0.0, float(vel_info['Y_STEP'])

def geolocation(x, y, geotrans):
    return [geotrans[0]+x*geotrans[1], geotrans[3]+y*geotrans[5]]

try:
    pnt_1 = geolocation(line.pnt1[0][0], line.pnt1[0][1], geotrans)
    pnt_2 = geolocation(line.pnt2[0][0], line.pnt2[0][1], geotrans)
    print(f"point 1: {pnt_1}")
    print(f"point 2: {pnt_2}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error may occur if a line was not selected.</text>'))

transection_args = f'{mint_path}/velocity.h5 -v {vel_vmin} {vel_vmax} --no-tight-layout --start-lalo {pnt_1[1]} {pnt_1[0]} --end-lalo {pnt_2[1]} {pnt_2[0]} --outfile x'

with osl.work_dir(plot_path):
    # let MintPy cleanup its old subplots 
    # this has been fixed in MintPy 1.5.3 but is required for 1.5.1
    plot_transection.main(transection_args.split()[:-2] + ['--noverbose', '--nodisplay'])
    # plot transection after removing old subplots
    %matplotlib inline
    plot_transection.main(transection_args.split())

---
## 18. Plot the Cumulative Displacement Map and Point Displacement Time Series

- Use the `Time` bar below the Cumulative Displacement Map to view displacements for different time periods
- Click on the Cumulative Displacement Map to select points for displaying Point Displacement Time-Series

In [ ]:
%matplotlib widget

tsview.main(
    [
        str(ts_demErr_sel_path),
        f'-d={mint_path}/inputs/geometryGeo.h5', 
        f'-o={mint_path}/displacement_ts',
        f'--outfile={mint_path}/displacement_ts.pdf'
    ]
)